<a href="https://colab.research.google.com/github/Graziottomp/ImageFileConvertion/blob/main/ICEYEkml2tiff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# Rotina para converter arquivos ICEYE em png e KML para imagens GeoTiff:

#author = 'Maria Paula Graziotto'

# copyright = "Copyright 2023, Maria Paula Graziotto"

# credits = ['Graziotto, M. P.']

# license = "Public Domain"

# version = "4.0"

# maintainer = "Maria Paula Graziotto"

# email = 'graziotto.mp@outlook.com'
#---------------------------------------------------------------------------------------

import pathlib
import pathlib as plib
from osgeo import ogr, gdal
import geotable
import numpy as np
import glob
import rasterio as rio
from rasterio.plot import show

def extract_coordinates(kml_file):
    t = geotable.load(kml_file, drop_z=True)
    poly = t.geometries[0].wkt.rsplit(sep=',')

    n = 5
    len_poly = len(poly)
    splited = [poly[i * len_poly // n:(i + 1) * len_poly // n] for i in range(n)]
    splited1 = ' '.join(map(str, splited[0]))
    coord1 = splited1[10:]

    lon = [coord1[:18], splited[1][1:19], splited[2][1:19], splited[3][1:19]]
    lat = [coord1[19:], splited[1][20:], splited[2][19:], splited[3][20:]]

    return lon, lat

def define_bounds(lon, lat):
    lon_min = min(lon)
    lon_max = max(lon)
    lat_min = min(lat)
    lat_max = max(lat)

    south = np.float64(lat_min)
    north = np.float64(lat_max)
    east = np.float64(lon_min)
    west = np.float64(lon_max)

    return south, north, east, west

def create_geotiff(png_file, name, lon, lat):
    with rio.open(png_file, 'r') as img:
        img_data = img.read([1]).astype('uint16')
        bounds = rio.transform.from_bounds(*define_bounds(lon, lat), img.shape[2], img.shape[1])
        crs = '+proj=latlong'

        with rio.open('{}.tiff'.format(name), 'w',
                      driver='GTiff',
                      count=1,
                      height=img.shape[1],
                      width=img.shape[2],
                      dtype=img_data.dtype,
                      crs=crs,
                      transform=bounds) as dst:
            dst.write(img_data)

def main():
    current_dir = plib.Path.cwd()
    root_dir = current_dir.parents[]

    kml_files = glob.glob('*.kml')
    kml_files.sort()

    png_files = glob.glob('*.png')
    png_files.sort()

    for kml_file, png_file in zip(kml_files, png_files):
        name = kml_file.rsplit(".")[0]
        lon, lat = extract_coordinates(kml_file)
        create_geotiff(png_file, name, lon, lat)

if __name__ == "__main__":
    main()
